In [6]:
import pandas as pd
import numpy as np

In [3]:
data = 'wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv'
!wget $data

Prepended http:// to 'wget'
--2025-10-11 22:36:58--  http://wget/
Resolving wget (wget)... failed: nodename nor servname provided, or not known.
wget: unable to resolve host address ‘wget’
--2025-10-11 22:36:58--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8003::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘course_lead_scoring.csv’

course_lead_scoring 100%[===================>]  78.98K  --.-KB/s    in 0.06s   

2025-10-11 22:36:59 (1.21 MB/s) - ‘course_lead_scoring.csv’ saved [80876/80876]

FINISHED --2025-10-11 22:36:59--
Total wall clock time: 0.5s
Downloaded: 1 files, 79K in 0.06s (1.21 MB/s)


In [8]:
df = pd.read_csv('03-classification/course_lead_scoring.csv')

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1
...,...,...,...,...,...,...,...,...,...
1457,referral,manufacturing,1,NaN,self_employed,north_america,4,0.53,1
1458,referral,technology,3,65259.0,student,europe,2,0.24,1
1459,paid_ads,technology,1,45688.0,student,north_america,3,0.02,1
1460,referral,NaN,5,71016.0,self_employed,north_america,0,0.25,1


In [24]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [28]:
numeric_na = ['annual_income']
categorical_na = ['lead_source', 'industry' , 'employment_status', 'location']

df[numeric_na] = df[numeric_na].fillna(0.0)
df[categorical_na] = df[categorical_na].fillna('NA')
df

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NA,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NA,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1
...,...,...,...,...,...,...,...,...,...
1457,referral,manufacturing,1,0.0,self_employed,north_america,4,0.53,1
1458,referral,technology,3,65259.0,student,europe,2,0.24,1
1459,paid_ads,technology,1,45688.0,student,north_america,3,0.02,1
1460,referral,NA,5,71016.0,self_employed,north_america,0,0.25,1


In [29]:
df['industry'].mode()

0    retail
Name: industry, dtype: object

In [31]:
print(abs(df['interaction_count'].corr(df['lead_score'])))
print(abs(df['number_of_courses_viewed'].corr(df['lead_score'])))
print(abs(df['number_of_courses_viewed'].corr(df['interaction_count'])))
print(abs(df['annual_income'].corr(df['interaction_count'])))


0.009888182496913115
0.00487899835468123
0.023565222882888086
0.02703647240481435


In [42]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [32]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42 )
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42 )
y_train = df_train['converted'].values
y_val = df_val['converted'].values
y_full_train = df_full_train['converted'].values
y_test = df_test['converted'].values

del df_train['converted']
del df_val['converted']
del df_full_train['converted']
del df_test['converted']

In [40]:
mutal_information = {}
mutal_information_options = ['industry', 'location', 'lead_source', 'employment_status']
for feature in mutal_information_options:
    mutal_information[feature] = round(mutual_info_score(df_train[feature], y_train),2)

print(max(mutal_information, key=mutal_information.get))
mutal_information

lead_source


lead_source


{'industry': 0.01,
 'location': 0.0,
 'lead_source': 0.04,
 'employment_status': 0.01}

In [63]:
numeric_na = ['annual_income', 'number_of_courses_viewed', 'interaction_count', 'lead_score']
categorical_na = ['lead_source', 'industry' , 'employment_status', 'location']
dv = DictVectorizer(sparse=False)
train_dict = df_train.to_dict(orient='records')
val_dict = df_val.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)
X_val = dv.transform(val_dict)
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)


In [77]:
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_val)[:, 1]
decision = (y_pred >= 0.5)
diff = round((y_val.astype(int)==decision).mean(), 4)
diff



np.float64(0.6997)

In [81]:
least_diff_dict = {}
for feature in df_train.columns.to_list():
    df_train_copy = df_train.copy()
    df_val_copy = df_val.copy()
    del df_train_copy[feature]
    del df_val_copy[feature]
    train_dict_copy = df_train_copy.to_dict(orient='records')
    val_dict_copy = df_val_copy.to_dict(orient='records')

    dv_copy =  DictVectorizer(sparse=False)
    X_train_copy = dv_copy.fit_transform(train_dict_copy)
    X_val_copy = dv_copy.transform(val_dict_copy)

    model_copy = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model_copy.fit(X_train_copy, y_train)
    y_pred_copy = model_copy.predict_proba(X_val_copy)[:, 1]
    decision_copy = (y_pred_copy >= 0.5)
    least_diff_dict[feature]= abs(round(diff - (y_val==decision_copy).mean(), 3))

print(least_diff_dict)
print(min(least_diff_dict, key = least_diff_dict.get))



{'lead_source': np.float64(0.003), 'industry': np.float64(0.0), 'number_of_courses_viewed': np.float64(0.143), 'annual_income': np.float64(0.154), 'employment_status': np.float64(0.003), 'location': np.float64(0.01), 'interaction_count': np.float64(0.143), 'lead_score': np.float64(0.007)}
industry


In [92]:
C_list = [0.01, 0.1, 1, 10, 100]
C_dict = {}
for C in  C_list:
    model_copy = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)
    model_copy.fit(X_train, y_train)
    y_pred_copy = model_copy.predict_proba(X_val)[:, 1]
    decision_copy = (y_pred_copy >= 0.5)
    C_dict[C]= round((y_val==decision_copy).mean(),3)

print(C_dict)
print(max(C_dict, key = C_dict.get))

{0.01: np.float64(0.7), 0.1: np.float64(0.7), 1: np.float64(0.7), 10: np.float64(0.7), 100: np.float64(0.7)}
0.01
